<a href="https://colab.research.google.com/github/denisefavila/Tensorflow-VGG16/blob/master/analysis_predictions_prod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import csv
import numpy as np
import pandas as pd
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
import itertools

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def get_probs(row):
    row_dict = json.loads(row._source.get("extra"))
    row["timestamp"] = row._source.get("@timestamp")
    row["id"] = row_dict.get("itinerary_id")
    row["predicted_proba"] = row_dict.get("predicted_allocation_probability")
    return row

# Predictions - Model

In [0]:

# Load data

path = "/content/drive/My Drive/data/prediction2.json"

df = pd.read_json(path, lines=True)
print("Shape: ", df.shape)



ValueError: ignored

In [0]:
'''
df.head()
'''

'\ndf.head()\n'

In [0]:
# json.loads(df._source[0].get("extra"))

In [0]:
''' 

tqdm.pandas()

df = df.progress_apply(get_probs, axis=1)

'''

' \n\ntqdm.pandas()\n\ndf = df.progress_apply(get_probs, axis=1)\n\n'

In [0]:
# data.to_csv("/content/drive/My Drive/data/prediction2.csv")

In [0]:
# Load data

path = "/content/drive/Team Drives/allocation_model/prediction2.csv"

df = pd.read_csv(path)
print("Shape: ", df.shape)

Shape:  (213838, 4)


In [0]:
data = df[["timestamp", "id", "predicted_proba"]]

In [0]:
data.shape

(213838, 3)

In [0]:
# Number of duplicated predictions (exactly the same)
data.duplicated().value_counts()

False    213214
True        624
dtype: int64

In [0]:
# Drop duplicated lines
data = data.drop_duplicates()

In [0]:
# Drop duplicated predictions to the same itinerary - keep the first one
data = data.drop_duplicates(subset=["id"], keep="first")


In [0]:
data.shape

(121695, 3)

In [0]:
print(data.timestamp.max()),
print(data.timestamp.min())

2019-05-22T22:59:54.310Z
2019-05-15T21:05:15.142Z


In [0]:
# Drop old itineraries (for some reason we have predictions to them)
data = data[data["id"] >= 13500000]

In [0]:
print("Final examples:", data.shape)

Final examples: (120904, 3)


In [0]:
print("Range total: ", data["id"].max() - data["id"].min())
print("Percentage of itineraries with preditions: ", data.shape[0]/float(data["id"].max() - data["id"].min()) )

Range total:  280443
Percentage of itineraries with preditions:  0.4311179098782997


# Status itinerary

In [0]:
# Load data

path = "/content/drive/Team Drives/allocation_model/itinerary_status_1505-2305.csv"

status = pd.read_csv(path, names=["itinerary_id", "product", "created", "status", 
                                  "accepted", "dropped", "finished"])
print("Shape: ", df.shape)

Shape:  (213838, 4)


In [0]:
status.head()

,itinerary_id,product,created,status,accepted,dropped,finished
0,13590416,2,2019-05-15 00:00:22.012744,cancelled,None,None,2019-05-15 00:00:25.180312
1,13590412,2,2019-05-15 00:01:20.488432,cancelledWithCharge,2019-05-15 00:04:13.368203,None,2019-05-15 00:18:43.023896
2,13590399,0,2019-05-15 00:05:29.216773,cancelled,None,None,2019-05-15 00:06:57.795438
3,13590425,2,2019-05-15 00:11:01.838824,cancelled,None,None,2019-05-15 00:11:26.438296
4,13590354,2,2019-05-15 00:16:16.890267,cancelled,None,None,2019-05-15 00:18:37.373659


In [0]:
status = status.merge(data[["id", "predicted_proba"]],
                      how="left",
                      left_on=["itinerary_id"],
                      right_on=["id"]).drop(columns=["id"])

In [0]:
status.head()

,itinerary_id,product,created,status,accepted,dropped,finished,predicted_proba
0,13590416,2,2019-05-15 00:00:22.012744,cancelled,None,None,2019-05-15 00:00:25.180312,NaN
1,13590412,2,2019-05-15 00:01:20.488432,cancelledWithCharge,2019-05-15 00:04:13.368203,None,2019-05-15 00:18:43.023896,0.572635
2,13590399,0,2019-05-15 00:05:29.216773,cancelled,None,None,2019-05-15 00:06:57.795438,NaN
3,13590425,2,2019-05-15 00:11:01.838824,cancelled,None,None,2019-05-15 00:11:26.438296,NaN
4,13590354,2,2019-05-15 00:16:16.890267,cancelled,None,None,2019-05-15 00:18:37.373659,NaN


In [0]:
status["prediction_flag"] = np.where((status["predicted_proba"].notnull()),
                                    1, 0)

In [0]:
status.head()

,itinerary_id,product,created,status,accepted,dropped,finished,predicted_proba,prediction_flag
0,13590416,2,2019-05-15 00:00:22.012744,cancelled,None,None,2019-05-15 00:00:25.180312,NaN,0
1,13590412,2,2019-05-15 00:01:20.488432,cancelledWithCharge,2019-05-15 00:04:13.368203,None,2019-05-15 00:18:43.023896,0.572635,1
2,13590399,0,2019-05-15 00:05:29.216773,cancelled,None,None,2019-05-15 00:06:57.795438,NaN,0
3,13590425,2,2019-05-15 00:11:01.838824,cancelled,None,None,2019-05-15 00:11:26.438296,NaN,0
4,13590354,2,2019-05-15 00:16:16.890267,cancelled,None,None,2019-05-15 00:18:37.373659,NaN,0


In [0]:
final_date = pd.to_datetime("2019-05-22T19:59")
initial_date = pd.to_datetime("2019-05-15T18:05")

status["created"] = pd.to_datetime(status["created"])

status.loc[(status["created"] >= initial_date) & (status["created"] <= final_date)].groupby("status").count()

In [0]:
# Discard Pro
status = status.loc[status["product"] != 1]

In [0]:
# Get only itineraries with predictions
status = status.loc[status["predicted_proba"].notnull()]

In [0]:
# Predictinos are wrong in production - we are getting the dropped 
status["predicted_proba"] = 1 - status["predicted_proba"]

In [0]:
status.head()

,itinerary_id,product,created,status,accepted,dropped,finished,predicted_proba,prediction_flag
1,13590412,2,2019-05-15 00:01:20.488432,cancelledWithCharge,2019-05-15 00:04:13.368203,None,2019-05-15 00:18:43.023896,0.427365,1
118,13590543,2,2019-05-15 07:26:37.209134,finished,2019-05-15 07:28:10.710294,None,2019-05-15 09:47:16,0.461308,1
124,13590530,2,2019-05-15 07:30:45.932463,cancelledWithCharge,2019-05-15 07:37:25.066506,None,2019-05-15 13:16:17.890432,0.281285,1
268,13590697,2,2019-05-15 07:49:29.513487,finished,2019-05-15 07:55:32.262150,None,2019-05-15 08:38:19,0.322187,1
368,13590805,2,2019-05-15 07:59:36.022221,finished,2019-05-15 07:59:58.363166,None,2019-05-15 15:07:13,0.412232,1


In [0]:
status["status"].value_counts()

finished               107813
cancelled                2904
cancelledWithCharge      2667
dropped                   315
merged                     44
Name: status, dtype: int64

In [0]:
status.shape

(113743, 9)

In [0]:
# I'm defining "allocated itineraries" as having an "accepted" date. 
# In training, we considered only "finished" and "dropped" itineraries (dropped = 0, finished = 1)
# We need to define how we will evaluate the canceled and cancelledWithCharge itineraries.

status["allocated_flag"] = np.where((status["accepted"] != "None"),
                                    1, 0)

In [0]:
# set the threshold >= 0.5
status["prediction"] = np.where((status["predicted_proba"] >= 0.50),
                                    1, 0)

In [0]:
status.head()

,itinerary_id,product,created,status,accepted,dropped,finished,predicted_proba,prediction_flag,prediction
1,13590412,2,2019-05-15 00:01:20.488432,cancelledWithCharge,2019-05-15 00:04:13.368203,None,2019-05-15 00:18:43.023896,0.427365,1,0
118,13590543,2,2019-05-15 07:26:37.209134,finished,2019-05-15 07:28:10.710294,None,2019-05-15 09:47:16,0.461308,1,0
124,13590530,2,2019-05-15 07:30:45.932463,cancelledWithCharge,2019-05-15 07:37:25.066506,None,2019-05-15 13:16:17.890432,0.281285,1,0
268,13590697,2,2019-05-15 07:49:29.513487,finished,2019-05-15 07:55:32.262150,None,2019-05-15 08:38:19,0.322187,1,0
368,13590805,2,2019-05-15 07:59:36.022221,finished,2019-05-15 07:59:58.363166,None,2019-05-15 15:07:13,0.412232,1,0


In [0]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [0]:
from sklearn.metrics import confusion_matrix

class_names = ["non_allocated","allocated"]


# Compute confusion matrix
cnf_matrix = confusion_matrix(status["allocated_flag"],
                              status["prediction"])

np.set_printoptions(precision=2)


# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=False,
                      title='Normalized confusion matrix')

plt.show()

KeyError: ignored

In [0]:
223/4454

0.05006735518634935